In [1]:
import asyncio
import json
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

In [ ]:
# server.py
from mcp.server.fastmcp import FastMCP

# 1. Instantiate the server
mcp = FastMCP(
    "DemoServer",
    host="127.0.0.1",
    port=8000,
    streamable_http_path="/mcp"  # Configure path in constructor
)

# 2. Define a tool that sums two numbers
@mcp.tool()
def add(a: int, b: int) -> int:
    """Return the sum of a and b."""
    return a + b

# 3. Run the server (defaults to HTTP on /mcp)
# 3️⃣ Run over Streamable HTTP (not STDIO), which spins up a Uvicorn server
if __name__ == "__main__":
    mcp.run(
        transport="streamable-http"    # Use HTTP transport instead of STDIO :contentReference[oaicite:0]{index=0}
                                        # host, port, and path removed from here
    )

TypeError: FastMCP.run() got an unexpected keyword argument 'host'

In [ ]:
async with streamablehttp_client("http://localhost:8000/mcp") as (read_stream, write_stream, _):
    async with ClientSession(read_stream, write_stream) as session:
        await session.initialize()
        result    = await session.call_tool(tool_name, arguments=params)

In [ ]:
    # 6. Invoke the MCP tool asynchronously
    try:
        result = await session.call_tool(tool_name, arguments=params)
        print(f"Result from '{tool_name}':\n{result}")
    except MCPError as e:
        print(f"MCP Error ({e.code}): {e.message}")


In [ ]:

def main():
    # 1. Initialize the OpenAI LLM and the MCP client
    llm = OpenAI(api_key="OPENAI_API_KEY") # Removed model="gpt-4"
    mcp = MCPClient(base_url="http://localhost:8000", api_key="MCP_API_KEY")

    # 2. User request that requires reading a file via MCP
    user_request = "Please show me the first 5 lines of /var/log/syslog"

    # 3. Instruct the model to output a JSON-formatted tool call when needed
    system_prompt = (
        "You are an assistant that can call MCP tools via JSON-RPC. "
        "When you need to use a tool, respond *only* with a JSON object "
        "containing 'tool' and 'params', and nothing else."
    )
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_request}
    ]

    # 4. Ask the model what tool to call
    response = llm.chat.completions.create(model="gpt-4", messages=messages) # Updated API call
    assistant_message = response.choices[0].message.content.strip()

    # 5. Parse the model’s JSON response
    call = json.loads(assistant_message)
    tool_name = call["tool"]
    params = call.get("params", {})

    # 6. Invoke the MCP server
    try:
        result = mcp.invoke(tool_name, params)
        print(f"Result from '{tool_name}':\n{result}")
    except MCPError as e:
        print(f"MCP Error ({e.code}): {e.message}")

if __name__ == "__main__":
    main()
